利用opencv-python从网络摄像头采集视频

In [1]:
import cv2

# 0输入参数指定网络摄像头的ID
cap = cv2.VideoCapture(0)
# 定义网络摄像头采集图像的比例系数
scaling_factor = 0.5
# 启动一个无限循环采集帧，直到按下Esc键
while True:
    ret, frame = cap.read()
    # 调整帧的大小
    frame = cv2.resize(frame, None, fx=scaling_factor, fy=scaling_factor, 
            interpolation=cv2.INTER_AREA)
    # 显示帧
    cv2.imshow('Webcam', frame)
    # 等待1ms，然后采集下一帧
    c = cv2.waitKey(1)
    if c == 27:
        break

# 释放视频采集对象
cap.release()
# 在结束代码之前关闭所有活动窗体
cv2.destroyAllWindows()

**用Haar级联创建一个人脸识别器**    
Haar级联通过在多个尺度上从图像中提取大量的简单特征来实现。简单特征主要指边、线、矩形特征等。
这些特征都非常易于计算，然后通过创建一系列简单的分类器来训练。
使用自适应增强技术可以使得这个过程更健壮。

In [3]:
import cv2
import numpy as np

# 加载人脸检测的级联文件
face_cascade = cv2.CascadeClassifier('cascade_files/haarcascade_frontalface_alt.xml')
# 确定级联文件是否正确地加载
if face_cascade.empty():
    raise IOError('Unable to load the face cascade classifier xml file')
    
# 生成一个视频采集对象
cap = cv2.VideoCapture(0)
scaling_factor = 0.5
while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, None, fx=scaling_factor, fy=scaling_factor, 
            interpolation=cv2.INTER_AREA)
    # 转为灰度图
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # 在灰度图上运行人脸检测器。
    # 参数1.3是指每个阶段的乘积系数。参数5是指每个候选矩形应该拥有的最小近邻数量。候选矩形是指人脸可能被检测到的候选区域
    face_rects = face_cascade.detectMultiScale(gray, 1.3, 5)
    # 在脸部画出矩形
    for (x,y,w,h) in face_rects:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 3)
    cv2.imshow('Face Detector', frame)
    c = cv2.waitKey(1)
    if c == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

下面用这种方法检测视频中的眼睛和鼻子

In [5]:
import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier('cascade_files/haarcascade_frontalface_alt.xml')
eye_cascade = cv2.CascadeClassifier('cascade_files/haarcascade_eye.xml')
nose_cascade = cv2.CascadeClassifier('cascade_files/haarcascade_mcs_nose.xml')

if face_cascade.empty():
    raise IOError('Unable to load the face cascade classifier xml file')
if eye_cascade.empty():
    raise IOError('Unable to load the eye cascade classifier xml file')
if nose_cascade.empty():
    raise IOError('Unable to load the nose cascade classifier xml file')
    
cap = cv2.VideoCapture(0)
scaling_factor = 0.5
while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, None, fx=scaling_factor, fy=scaling_factor, 
            interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        # 从彩色和灰度图中提取人脸ROI信息
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        # 从灰度图ROI信息中检测眼睛
        eye_rects = eye_cascade.detectMultiScale(roi_gray)
        # 检测鼻子
        nose_rects = nose_cascade.detectMultiScale(roi_gray, 1.3, 5)
        # 在眼睛周围画绿色的圆
        for (x_eye, y_eye, w_eye, h_eye) in eye_rects:
            center = (int(x_eye + 0.5*w_eye), int(y_eye + 0.5*h_eye))
            radius = int(0.3 * (w_eye + h_eye))
            color = (0, 255, 0)
            thickness = 3
            cv2.circle(roi_color, center, radius, color, thickness)
        # 在鼻子周围画矩形
        for (x_nose, y_nose, w_nose, h_nose) in nose_rects:
            cv2.rectangle(roi_color, (x_nose, y_nose), (x_nose+w_nose, 
                y_nose+h_nose), (0,255,0), 3)
            break
    cv2.imshow('Eye and nose detector', frame)
    c = cv2.waitKey(1)
    if c == 27:
        break
            
cap.release()
cv2.destroyAllWindows()        